In [35]:
import re
import pandas as pd
import numpy as np
import joblib

In [2]:
old = pd.read_csv('perbandingan/berita-24-5.csv')
# print(old)
new = pd.read_csv('perbandingan/berita-24-5.csv') # need changes
# print(new)

In [4]:
# old.columns
old.columns

Index(['ID', 'berita_title', 'berita_date', 'berita_qdate', 'berita_desc',
       'berita_source', 'created_at', 'berita_topik_id', 'qe_what', 'ner_when',
       'ner_who', 'ner_prov', 'ner_kab', 'ner_kec', 'ner_neg', 'sev_death',
       'sev_injury', 'sev_lost', 'sev_sink', 'sev_plane', 'sev_moneyUSD',
       'sev_moneyIDR', 'sev_persen', 'sev_inflasi', 'sev_internasional',
       'sev_nasional', 'sev_provkota', 'sev_lost_items', 'sev_lost_money',
       'sev_rape', 'sev_house', 'sev_person', 'class_classification',
       'berita_summary'],
      dtype='object')

# Comparing What

In [13]:
qe_old = old[['ID', 'berita_title', 'berita_topik_id', 'qe_what']]
# qe_old.head()
qe_new = new[['ID', 'berita_title', 'berita_topik_id', 'qe_what']] # possibly need changes
# qe_old.head()

# Comparing Who

# Comparing When

# Comparing Where

# Comparing Classification 

In [9]:
cls_old = old[['ID', 'berita_title', 'berita_topik_id', 'sev_death', \
       'sev_injury', 'sev_lost', 'sev_sink', 'sev_plane', 'sev_moneyUSD', \
       'sev_moneyIDR', 'sev_persen', 'sev_inflasi', 'sev_internasional', \
       'sev_nasional', 'sev_provkota', 'sev_lost_items', 'sev_lost_money', \
       'sev_rape', 'sev_house', 'sev_person', 'class_classification']]

In [10]:
cls_old.head(1)

,ID,berita_title,berita_topik_id,sev_death,sev_injury,sev_lost,sev_sink,sev_plane,sev_moneyUSD,sev_moneyIDR,...,sev_inflasi,sev_internasional,sev_nasional,sev_provkota,sev_lost_items,sev_lost_money,sev_rape,sev_house,sev_person,class_classification
0,1,Bagaimana Proses Terjadinya Gempa Bumi?,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,sedang


In [40]:
# Bencana Section (Topic id = 1)
cls_old_bencana = cls_old[cls_old['berita_topik_id'] == 1]
cls_old_bencana = cls_old_bencana[['sev_death', 'sev_injury', 'sev_house', 'sev_person']]
# print(len(cls_old_bencana))
cls_old_bencana.rename(columns = {'sev_death':'mati', 'sev_injury':'luka', 'sev_house':'rumah', 'sev_person':'orang'}, inplace=True)
cls_old_bencana.dtypes

mati     float64
luka     float64
rumah    float64
orang    float64
dtype: object

In [41]:
# cls_old_bencana['mati'] = [re.sub("[^0-9]", "", str(mati)) if (re.sub("[^0-9]", "", str(mati)) != '') else 1 for mati in cls_old_bencana['mati']]
cls_old_bencana['mati'] = cls_old_bencana['mati'].astype(int)

# cls_old_bencana['luka'] = [re.sub("[^0-9]", "", str(luka)) if (re.sub("[^0-9]", "", str(luka)) != '') else 1 for luka in cls_old_bencana['luka']]
cls_old_bencana['luka'] = cls_old_bencana['luka'].astype(int)

# cls_old_bencana['rumah'] = [re.sub("[^0-9]", "", str(rumah)) if (re.sub("[^0-9]", "", str(rumah)) != '') else 1 for rumah in cls_old_bencana['rumah']]
cls_old_bencana['rumah'] = cls_old_bencana['rumah'].astype(int)

# cls_old_bencana['orang'] = [re.sub("[^0-9]", "", str(orang)) if (re.sub("[^0-9]", "", str(orang)) != '') else 1 for orang in cls_old_bencana['orang']]
cls_old_bencana['orang'] = cls_old_bencana['orang'].astype(int)
# cls_old_bencana = cls_old_bencana.replace({',': '.'}, regex=True)

cls_old_bencana.head()

,mati,luka,rumah,orang
0,0,0,0,0
1,0,0,5,0
2,1,7729,1,318
3,0,1,0,1
4,0,0,25,230


In [30]:
cls_old_bencana[['class_classification']].value_counts()

class_classification
sedang                  1968
parah                    805
dtype: int64